In [1]:
%%capture

import json
import tqdm
import nltk

nltk.download('punkt')

In [2]:
dataset_id = 0
datasets = ['peerread', 'acl', 'refseer', 'arxiv']
dataset = datasets[dataset_id]

In [3]:
with open('./' + dataset + '/' + 'papers.json', 'r+') as f:
    papers = json.load(f)
    
with open('./' + dataset + '/' + 'contexts.json', 'r+') as f:
    contexts = json.load(f)

mapping_types = ['train', 'val', 'test']
mapping_type_id = 0
mapping_type = mapping_types[mapping_type_id]

In [4]:
def process(text):
    tokens = [token for token in nltk.word_tokenize(text) if token != ' '
                                                          and token != 'OTHERCIT'
                                                          and token != 'mcOTHERCIT']
    return tokens

def isolate_sentence(text):
    sents = nltk.sent_tokenize(text)
    main_sent = ''
    for sent in sents:
        if 'TARGETCIT' in sent:
            main_sent = sent
            break
            
    tokens = [token for token in nltk.word_tokenize(main_sent) if token != ''
                                                               and token != 'TARGETCIT'
                                                               and token != 'OTHERCIT']
    return tokens

def tokens_to_string(tokens):
    return ' '.join(tokens).strip()

In [5]:
def fill_store(datapoints, papers, contexts):
    global store
    for unit in tqdm.tqdm(datapoints):
        context_id = unit['context_id']
        paper_ids = unit['positive_ids']

        paper_node = papers[paper_ids[0]]
        context_text = contexts[context_id]['masked_text']

        blob = {}
        blob['cite_span'] = tokens_to_string(isolate_sentence(context_text))
        blob['cite_context'] = tokens_to_string(process(context_text))
        blob['raw_context_id'] = context_id
        blob['paper_id'] = paper_ids[0]

        blob['paper'] = paper_node

        store.append(blob)

In [6]:
# blob['paper'] fields (for all four datasets):
# 'title'
# 'abstract'
# 'authors'

In [7]:
store = []

with open('./' + dataset + '/' + mapping_types[0] + '.json', 'r+') as f:
    set_A = json.load(f)

with open('./' + dataset + '/' + mapping_types[1] + '.json', 'r+') as f:
    set_B = json.load(f)
    
fill_store(set_A, papers, contexts)
fill_store(set_B, papers, contexts)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 492/492 [00:00<00:00, 3216.63it/s]


In [8]:
with open('./' + dataset + '/' + 'index.json', 'w+') as f:
    json.dump(store, f)

In [9]:
# That's it